# Import necessary packages

In [1]:
# Import necessary packages
import os
import sys
import math
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import warnings
from pathlib import PurePath

# Ignore warnings
warnings.filterwarnings("ignore")

# 1. Data Preprocessing

Data preprocessing refers to the process of cleaning, transforming, and preparing raw data before conducting data analysis. The following are several important functions of data preprocessing:

- Data cleaning: Data preprocessing involves identifying and handling outliers, missing values, duplicate values, or erroneous values in the data. Cleaning the data helps ensure data quality and accuracy, avoiding adverse effects on the analysis results.
- Data transformation: Through data preprocessing, data can be transformed into formats or representations suitable for specific analysis methods. For example, transformation operations such as feature scaling, standardization, normalization, or discretization can be performed to obtain better results in subsequent analysis.
- Data integration: When data comes from different sources or different data tables, data preprocessing can help integrate them into a consistent dataset for analysis and modeling purposes.
- Handling missing values: Missing values are common in real-world data. Data preprocessing can utilize imputation methods such as mean imputation, median imputation, or regression imputation to handle missing values, thereby reducing data loss and analysis bias.
- Tagging as 'internal testing period': the internal testing period refers to a short interval between informal testing and formal testing, during which the releases are labeled as NA and cannot be filled.

By performing data preprocessing, data quality, accuracy, and consistency can be improved, providing a more reliable foundation for subsequent quantitative and qualitative analysis tasks. It helps reduce analysis errors, enhance model performance, and ensure meaningful conclusions and insights are derived from the data.

The data being processed in this case primarily consists of two parts: true release data and report event data.

## 1.1 Preprocessing for True Release Data

In order to facilitate subsequent analysis tasks, the real release dose data processing in this project involves the following steps:

1. Detect and interpolate missing values in methane flow rate data using adjacent time points
2. Organize the data daily, spanning from 00:00:00 to 23:59:59 UTC
3. Mark internal testing periods as N/A
4. tag all true release: 
    - "0" indicates that the original release data
    - "1" indicates that the data represents the interval testing, and the True Release rate (kg/h) is set as NA.
    - "2" indicates that the data with True Release rate (kg/h) as NA has been replaced with linearly interpolated data, usually for the middle time of each day.
    - "3" indicates nightly padding, where the start and end of the raw data are extended to the current 00:00:00 and 23:59:59, and the True Release rate (kg/h) is set as 0.

These steps are performed to ensure the completeness and continuity of the real release dose data for further analysis. By filling in missing values and creating a continuous timeline, we can facilitate accurate analysis and interpretation of the data.



In [2]:
# Set the data path
data_path = PurePath("../../assets/Raw_Data_Per_Day")

# Get all the files of true release data and sort them
filenames = []
for name in os.listdir(data_path):
    filenames.append(name)

# Function used to sort file names
def filenames_sort_map(name):
    """
    Args:
        name: str, file name
        
    Returns:
        str, sorted file name
    """
    s, _ = os.path.splitext(name)
    m, d = s.split("_")
    return "%02d-%02d" %(int(m), int(d))


filenames = sorted(filenames, key=filenames_sort_map)
precess_na_number = 0

# 1. Handling missing values in the middle of each day
valid_true_data = None
for filename in filenames:
    print(filename)
    file_path = PurePath(data_path, filename)
    data_t = pd.read_csv(file_path)
    t_s = data_t.loc[0, "datetime_utc"]
    t_e = data_t.loc[data_t.index[-1], "datetime_utc"]

    # Check if time is continuous
    assert (pd.Timestamp(t_e) - pd.Timestamp(t_s)) / pd.Timedelta(1, "s") == data_t.shape[0] - 1, print(t_e, t_s, data_t.shape[0])
    
    # add tag column for data_t, default value is 0, which means original data, 
    # 1 means internal testing period, 2 means linear interpolation, 3 means nightly padding
    data_t["tag"] = 0
    print("\tOrigin data length: ", data_t.shape[0])
    print("\tOrigin NA number: {}, not NA number: {}".format(data_t["methane_kgh"].isna().sum(), data_t["methane_kgh"].notna().sum()))
    prev_na_number = data_t["methane_kgh"].isna().sum()
    prev_notna_number = data_t["methane_kgh"].notna().sum()


    # For the middle of each day (from the first non-NA to the last non-NA), use linear interpolation to fill in missing values, and mark the filled NA values as 2
    not_na_index = data_t.loc[data_t["methane_kgh"].notna(), :].index
    start_not_na = not_na_index[0]
    end_not_na = not_na_index[-1]
    mid_idx = np.zeros(data_t.shape[0])
    mid_idx[start_not_na:end_not_na+1] = 1
    data_t.loc[mid_idx & (data_t["methane_kgh"].isna()), "tag"] = 2 # 2 means linear approximation
    data_t.loc[start_not_na:end_not_na, "methane_kgh"] = data_t.loc[start_not_na:end_not_na, "methane_kgh"].interpolate()
    precess_na_number += (data_t["tag"] == 2).sum()

    print("\tAfter linear interpolation, NA number: {}, not NA number: {}".format(
        data_t["methane_kgh"].isna().sum(), 
        data_t["methane_kgh"].notna().sum()))

    data_t = data_t.rename(columns={"datetime_utc": "Datetime (UTC)", "methane_kgh": "Release Rate (kg/h)"})
    data_t = data_t[["Datetime (UTC)", "Release Rate (kg/h)", "tag"]]


    # Merge the data for each day together
    if valid_true_data is None:
        valid_true_data = data_t
    else:
        valid_true_data = pd.concat([valid_true_data, data_t], axis=0)
    
print("total processed NA number: ", precess_na_number)

10_10.csv
	Origin data length:  19801
	Origin NA number: 1697, not NA number: 18104
	After linear interpolation, NA number: 1667, not NA number: 18134
10_11.csv
	Origin data length:  26940
	Origin NA number: 951, not NA number: 25989
	After linear interpolation, NA number: 793, not NA number: 26147
10_12.csv
	Origin data length:  77101
	Origin NA number: 56714, not NA number: 20387
	After linear interpolation, NA number: 0, not NA number: 77101
10_13.csv
	Origin data length:  6460
	Origin NA number: 2461, not NA number: 3999
	After linear interpolation, NA number: 2449, not NA number: 4011
10_14.csv
	Origin data length:  2081
	Origin NA number: 21, not NA number: 2060
	After linear interpolation, NA number: 0, not NA number: 2081
10_17.csv
	Origin data length:  10903
	Origin NA number: 2418, not NA number: 8485
	After linear interpolation, NA number: 2408, not NA number: 8495
10_18.csv
	Origin data length:  4201
	Origin NA number: 1277, not NA number: 2924
	After linear interpolation, 

In [3]:
# 2. Handling missing values at the beginning and end of each day
# Fill in 0 values for the beginning and end of each day, and ensure data continuity, marked padding data as 3
valid_true_data["Datetime (UTC)"] = pd.to_datetime(valid_true_data["Datetime (UTC)"])
valid_true_data.sort_values(by="Datetime (UTC)", inplace=True)
valid_true_data.reset_index(drop=True, inplace=True)
timeInterval = pd.Timestamp(valid_true_data.loc[valid_true_data.shape[0]-1, "Datetime (UTC)"].date()) + pd.Timedelta(1, "d") - pd.Timestamp(valid_true_data.loc[0, "Datetime (UTC)"].date())
data_len = timeInterval / pd.Timedelta(1, "s")
new_valid_true_data = pd.DataFrame(columns=["Datetime (UTC)", "True Release Rate (kg/h)"])
start_time = pd.Timestamp(valid_true_data.loc[0, "Datetime (UTC)"].date())

new_datetime = []
for i in tqdm(range(0, int(data_len))):
    new_datetime.append(start_time + pd.Timedelta(i, "s"))
new_valid_true_data["Datetime (UTC)"] = new_datetime
new_valid_true_data["True Release Rate (kg/h)"] = 0
new_valid_true_data["tag"] = 3 # 3 means night padding

true_data_indexes = []
for i in tqdm(range(valid_true_data.shape[0])):
    if pd.notna(valid_true_data.loc[i, "Release Rate (kg/h)"]):
        true_data_indexes.append(
            int((valid_true_data.loc[i, "Datetime (UTC)"] - start_time) / pd.Timedelta(1, "s"))
        )
new_valid_true_data.loc[true_data_indexes, "True Release Rate (kg/h)"] = valid_true_data.loc[valid_true_data["Release Rate (kg/h)"].notna(), "Release Rate (kg/h)"].tolist()
# merge tag column
new_valid_true_data.loc[true_data_indexes, "tag"] = valid_true_data.loc[valid_true_data["Release Rate (kg/h)"].notna(), "tag"].tolist()

print("After continuous processing, interpolation number is {}, nightly padding number is {}, origin number is {}".format(
    new_valid_true_data.loc[new_valid_true_data["tag"] == 2, "tag"].shape[0],
    new_valid_true_data.loc[new_valid_true_data["tag"] == 3, "tag"].shape[0],
    new_valid_true_data.loc[new_valid_true_data["tag"] == 0, "tag"].shape[0]
))

100%|██████████| 713975/713975 [00:19<00:00, 37209.78it/s]


After continuous processing, interpolation number is 123041, nightly padding number is 3819457, origin number is 550302


In [5]:
# Tagging as 'internal testing period', marked as 1
testing_period_data_path = PurePath("../../assets/Daily Test Cycle.xlsx")
testing_period_data = pd.read_excel(testing_period_data_path)
testing_period_data.dropna(axis=0, how="all", inplace=True)

# Convert the time format of the testing period data to the same format as the data
columns = ["informal testing start",
           "informal testing end",
           "official testing start",
           "official testing end"]
for column in columns:
    testing_period_data[column] = pd.to_datetime(testing_period_data[column])



# extract the internal testing period
testing_period = []
for i in range(testing_period_data.shape[0]):
    informal_start = testing_period_data.loc[i, "informal testing start"]
    informal_end = testing_period_data.loc[i, "informal testing end"]
    official_start = testing_period_data.loc[i, "official testing start"]
    official_end = testing_period_data.loc[i, "official testing end"]

    if official_start > informal_start:
        testing_period.append([informal_start, official_start])
    if official_end < informal_end:
        testing_period.append([official_end, informal_end])
print("Processing Internal testing period:" )
for period in testing_period:
    start = period[0]
    end = period[1]
    print("Internal testing period: start: {}, end: {}".format(start, end), end="     ")
    na_number = new_valid_true_data["True Release Rate (kg/h)"].isna().sum()
    new_valid_true_data.loc[(new_valid_true_data["Datetime (UTC)"] >= start) & (new_valid_true_data["Datetime (UTC)"] <= end), "tag"] = 1
    new_valid_true_data.loc[(new_valid_true_data["Datetime (UTC)"] >= start) & (new_valid_true_data["Datetime (UTC)"] <= end), "True Release Rate (kg/h)"] = np.nan
    print("Ignored NA number: {}".format(new_valid_true_data["True Release Rate (kg/h)"].isna().sum() - na_number))

print("{} number of points have been filtered".format(new_valid_true_data.loc[new_valid_true_data["tag"] == 1, "tag"].shape[0]))

Processing Internal testing period:
Internal testing period: start: 2022-10-10 16:42:29, end: 2022-10-10 17:00:00     Ignored NA number: 1052
Internal testing period: start: 2022-10-10 21:36:00, end: 2022-10-11 00:11:29     Ignored NA number: 9330
Internal testing period: start: 2022-10-11 16:43:13, end: 2022-10-11 17:16:17     Ignored NA number: 1985
Internal testing period: start: 2022-10-11 21:16:36, end: 2022-10-12 00:02:42     Ignored NA number: 9967
Internal testing period: start: 2022-10-12 15:41:07, end: 2022-10-12 17:15:56     Ignored NA number: 5690
Internal testing period: start: 2022-10-12 21:00:00, end: 2022-10-12 23:00:00     Ignored NA number: 7201
Internal testing period: start: 2022-10-13 17:50:11, end: 2022-10-13 20:40:00     Ignored NA number: 10190
Internal testing period: start: 2022-10-13 21:08:00, end: 2022-10-14 00:26:01     Ignored NA number: 11882
Internal testing period: start: 2022-10-14 15:16:11, end: 2022-10-14 16:25:00     Ignored NA number: 4130
Internal

In [6]:
# Save continuous data
new_valid_true_data.to_csv(PurePath("../../assets/valid_true_data_pad.csv"), index=False)

# Save continuous data for each day
for date in pd.date_range(start=pd.to_datetime("2022-10-10").date(), end=pd.to_datetime("2022-11-30").date(), freq="d"):
    date_str = date.strftime("%Y-%m-%d")
    start_time = pd.Timestamp(date)
    end_time = pd.Timestamp(date) + pd.Timedelta(1, "d")
    data = new_valid_true_data.loc[(new_valid_true_data["Datetime (UTC)"] >= start_time) & (new_valid_true_data["Datetime (UTC)"] < end_time), :]
    if date in [
        pd.to_datetime("2022-10-14").date(),
        pd.to_datetime("2022-10-18").date(),
        pd.to_datetime("2022-10-19").date(),
        pd.to_datetime("2022-10-20").date(),
        pd.to_datetime("2022-10-21").date(),
        pd.to_datetime("2022-10-22").date()
    ]:
        data.loc[:, "tag"] = 1
        data.loc[:, "True Release Rate (kg/h)"] = np.nan

    print("Date: ", date_str)
    print("\tNumber of Original data:", (data["tag"] == 0).sum())
    print("\tNumber of Interpolation data:", (data["tag"] == 2).sum())
    print("\tNumber of Nightly padding data:", (data["tag"] == 3).sum())
    print("\tNumber of Ignored data:", (data["tag"] == 1).sum())

    data.reset_index(drop=True, inplace=True)
    data.to_csv(PurePath("../../assets/valid_true_data_pad_daily", "valid_true_data_pad_daily_" + date_str + ".csv"), index=False)

Date:  2022-10-10
	Number of Original data: 16545
	Number of Interpolation data: 14
	Number of Nightly padding data: 60149
	Number of Ignored data: 9692
Date:  2022-10-11
	Number of Original data: 14299
	Number of Interpolation data: 119
	Number of Nightly padding data: 59503
	Number of Ignored data: 12479
Date:  2022-10-12
	Number of Original data: 13430
	Number of Interpolation data: 56317
	Number of Nightly padding data: 3599
	Number of Ignored data: 13054
Date:  2022-10-13
	Number of Original data: 1670
	Number of Interpolation data: 9
	Number of Nightly padding data: 64211
	Number of Ignored data: 20510
Date:  2022-10-14
	Number of Original data: 0
	Number of Interpolation data: 0
	Number of Nightly padding data: 0
	Number of Ignored data: 86400
Date:  2022-10-15
	Number of Original data: 0
	Number of Interpolation data: 0
	Number of Nightly padding data: 86400
	Number of Ignored data: 0
Date:  2022-10-16
	Number of Original data: 0
	Number of Interpolation data: 0
	Number of Nigh

## 1.2 Preprocessing for Report Event Data

For the report event data, the data preprocessing includes the following steps:

1) Extraction of required data: The necessary data is parsed, including the event start time, end time, release dose, etc.
2) Extraction of Missing Report Date: The missing report dates are extracted from the data.
3) Extraction of Report Date: The report dates are extracted, which include the report start time and report end time.

These steps are performed to preprocess the report event data, extracting the relevant information needed for further analysis. By extracting the necessary data and identifying missing report dates, we can ensure the completeness and accuracy of the dataset for subsequent analysis and interpretation.

### 1.2.1 Extract valid event data for all sensors

In [7]:
# extract all sensor data 
DATA_TO_CSV = True

# Set the root path and file names
root = PurePath('../../assets/sensor_raw_data/')

valid_event_df = {}

names = ['Qube', 'Oiler', 'Sensirion', 'Canary','Kuva','Ecoteco','Andium', "Soofie"]
files = ['0222_Qube_DataReportingTemplate_Continuous Monitoring.xlsx',
         '0222_DataReportingTemplate_Continuous Monitoring _Oiler Equation.xlsx',
         'DataReporting_Stanford_SensirionConnectedSolutions_20230228.xlsx',
         'DataReportingTemplate_Project Canary_Continuous Monitoring_results.xlsx',
         'DataReportingKUVA_Continuous Monitoring-Final.xlsx',
         'DataReportingTemplate_Continuous  Monitoring_UTC_ECOTEC.xlsx',
         'Andium_DataReporting_Continuous Monitoring_Final_revised.xlsx', 
         "SOOFIE DataReportingTemplate_Continuous Monitoring updated 2022 06 13.xlsx"]

if DATA_TO_CSV:

    # 2. Read the data and save to csv
    for i in tqdm(range(len(names))):
        print("Processing {} data".format(names[i]))
        sensor_name = names[i]
        # Keep the valid columns
        path = PurePath(root, files[i])
        if names[i] == "Soofie":
            lab_data = pd.read_excel(path, sheet_name='Survey Summary', parse_dates=['EmissionStartDateTime UTC','EmissionEndDateTime UTC'])
            lab_data.rename(columns={'EmissionStartDateTime UTC':'EmissionStartDateTime', 'EmissionEndDateTime UTC':'EmissionEndDateTime'}, inplace=True)
        else:
            lab_data = pd.read_excel(path, sheet_name='Survey Summary', parse_dates=['EmissionStartDateTime','EmissionEndDateTime'])

        if names[i] == "Qube":
            lab_data["EmissionRateUpper"] /= 1000
            lab_data["EmissionRateLower"] /= 1000

        lab_data.dropna(subset=['EmissionStartDateTime','EmissionEndDateTime'], how='all', inplace=True)
        valid_cols = ['EmissionStartDateTime', 'EmissionEndDateTime', 'EmissionRate', "EmissionRateUpper", "EmissionRateLower"]
        lab_data = lab_data[valid_cols].copy()

        lab_data.rename(columns={'EmissionRate':'Reported Release Rate (kg/h)'}, inplace=True)
        lab_data.sort_values(by='EmissionStartDateTime', inplace=True)
        if not os.path.exists(PurePath('../../assets/sensor_data')):
            os.makedirs(PurePath('../../assets/sensor_data'))
        print("\t Original events number: ", lab_data.shape[0])
        valid_event_df[names[i]] = lab_data
        # Save to csv
        # save_path = PurePath('../../assets/sensor_data/%s_validdata.csv'%names[i])
        # lab_data.to_csv(save_path, index=None)

  0%|          | 0/8 [00:00<?, ?it/s]

Processing Qube data


 12%|█▎        | 1/8 [00:00<00:03,  1.91it/s]

	 Original events number:  116
Processing Oiler data


 25%|██▌       | 2/8 [00:01<00:03,  1.90it/s]

...

	 Original events number:  532
Processing Andium data


 88%|████████▊ | 7/8 [00:04<00:00,  1.52it/s]

	 Original events number:  164
Processing Soofie data


100%|██████████| 8/8 [00:05<00:00,  1.53it/s]

	 Original events number:  4510


### 1.2.2 Extract Offline Date
only Kuva and Oiler have Offline Date, we can extract the date from **DataReportingKUVA_Continuous Monitoring-Final.xlsx** and **0222_DataReportingTemplate_Continuous Monitoring _Oiler Equation.xlsx**

In [8]:
def get_offline_time(excel_file):
    tmp_df = pd.read_excel(excel_file, sheet_name="Missing Data Reporting")
    tmp_df = tmp_df[["OfflineReportStartDateTime", "OfflineReportEndDateTime"]]
    tmp_df["OfflineReportStartDateTime"] = pd.to_datetime(tmp_df["OfflineReportStartDateTime"])
    tmp_df["OfflineReportEndDateTime"] = pd.to_datetime(tmp_df["OfflineReportEndDateTime"])
    tmp_df = tmp_df.dropna(subset=["OfflineReportStartDateTime", "OfflineReportEndDateTime"], how="any")
    return tmp_df


print("Missing Date Reporting...")
# For Kuva 
kuva_offline_time = get_offline_time(PurePath(root, "DataReportingKUVA_Continuous Monitoring-Final.xlsx"))
kuva_offline_time.to_excel(PurePath("../../assets/sensor_data/Kuva_offline_data.xlsx"), index=False, sheet_name="Missing Data Reporting")
print("For Kuva, offline time number is {}".format(kuva_offline_time.shape[0]))

# For Oiler
oiler_offline_time = get_offline_time(PurePath(root, "0222_DataReportingTemplate_Continuous Monitoring _Oiler Equation.xlsx"))
oiler_offline_time.to_excel(PurePath("../../assets/sensor_data/Oiler_offline_data.xlsx"), index=False, sheet_name="Missing Data Reporting")
print("For Oiler, offline time number is {}".format(oiler_offline_time.shape[0]))

Missing Date Reporting...
For Kuva, offline time number is 51
For Oiler, offline time number is 14


### 1.2.3 Extract Online Report Days

The Online Report Days also called Deploy time, which include the report start time and report end time.

In [9]:
sensor_names = ['Qube', 'Oiler', 'Sensirion', 'Canary','Kuva','Ecoteco','Andium', "Soofie"]
online_date = pd.DataFrame(columns=["Sensor", "Testing Beginning Date", "Testing Ending Date"])
release_start_time = pd.to_datetime("2022-10-10 00:00:00")
release_end_time = pd.to_datetime("2022-11-30 23:59:59")

print("Online Date Reporting...")
for i in range(len(sensor_names)):
    print("Processing {} data".format(sensor_names[i]))
    tmp_df = valid_event_df[sensor_names[i]]
    tmp_df["EmissionStartDateTime"] = pd.to_datetime(tmp_df["EmissionStartDateTime"])
    tmp_df["EmissionEndDateTime"] = pd.to_datetime(tmp_df["EmissionEndDateTime"])
    tmp_df = tmp_df[(tmp_df["EmissionStartDateTime"] >= release_start_time) & (tmp_df["EmissionEndDateTime"] <= release_end_time)]
    tmp_df = tmp_df[["EmissionStartDateTime", "EmissionEndDateTime"]]
    print("\t Online time is {}->{}".format(tmp_df["EmissionStartDateTime"].min(), tmp_df["EmissionEndDateTime"].max()))
    online_date.loc[i] = [sensor_names[i], tmp_df["EmissionStartDateTime"].min(), tmp_df["EmissionEndDateTime"].max()]

online_date.to_excel(PurePath("../../assets/sensor_data/sensor_online_date.xlsx"), index=False, sheet_name="online_date")

Online Date Reporting...
Processing Qube data
	 Online time is 2022-10-10 16:50:00->2022-11-23 22:12:00
Processing Oiler data
	 Online time is 2022-10-10 18:03:35->2022-11-03 18:56:33
Processing Sensirion data
	 Online time is 2022-10-10 16:51:00->2022-11-30 18:44:00
Processing Canary data
	 Online time is 2022-10-10 16:54:00->2022-11-29 18:33:00
Processing Kuva data
	 Online time is 2022-10-10 16:51:00->2022-11-23 22:03:00
Processing Ecoteco data
	 Online time is 2022-10-28 16:56:07->2022-11-28 17:45:23
Processing Andium data
	 Online time is 2022-10-10 16:50:46->2022-11-23 18:36:34
Processing Soofie data
	 Online time is 2022-10-10 00:00:00->2022-11-29 23:30:00


## 1.2.4 Team data process

1. Set the report release rate for each event to N/A for (Andium/Ecoteco/Kuva);
2. Definition of Events:

    1) All sensors do not include SOOFIE
        1) Positive Events: Release rate > 0 kg/hr or labeled as N/A
        2) Negative Events: Release rate = 0 kg/hr
        3) Events between two consecutive events are classified as negative 
    2) Only for SOOFIE
        1) Positive Events: Release Rate > 0 kg/hr
        2) Negative Events: Release Rate = 0 kg/hr
        3) N/A Events: Release Rate is N/A

3. Event filtering:
    1) Canary: Short Stack Head Filtering, Exclude events that are not within the dates of October 31, 2022, and November 15 to November 30, 2022.
    2) Kuva: Exclude events overlapping with Offline Report Dates
    3) Oiler: Exclude events overlapping with Offline Report Dates
    4) SOOFIE: Exclude events falling within the period of November 7, 2022, 22:30:00 to November 14, 2022, 16:30:00

4. For each sensor, filter out all events that do not overlap with the sensor's Online Date
5. Event Removal: 
    - Exclude reports on: 2022-10-14, 2022-10-18 to 2022-10-22
    - Reason: Stanford internal testing period

In [10]:
# 1. Set the report release rate for each event to N/A for (Andium/Ecoteco/Kuva);

# For Andium
andium_df = valid_event_df["Andium"].copy()
andium_df["Reported Release Rate (kg/h)"] = np.nan
andium_df["EmissionRateUpper"] = np.nan
andium_df["EmissionRateLower"] = np.nan

# For Ecoteco
ecoteco_df = valid_event_df["Ecoteco"].copy()
ecoteco_df["Reported Release Rate (kg/h)"] = np.nan
ecoteco_df["EmissionRateUpper"] = np.nan
ecoteco_df["EmissionRateLower"] = np.nan

# For Kuva
kuva_df = valid_event_df["Kuva"].copy()
kuva_df["Reported Release Rate (kg/h)"] = np.nan
kuva_df["EmissionRateUpper"] = np.nan
kuva_df["EmissionRateLower"] = np.nan


# 2. Definition of Events:

#     1) All sensors exclude SOOFIE
#         1) Positive Events: Release rate > 0 kg/hr or labeled as N/A
#         2) Negative Events: Release rate = 0 kg/hr
#         3) Events between two consecutive events are classified as negative 
#     2) Only for SOOFIE
#         1) Positive Events: Release Rate > 0 kg/hr
#         2) Negative Events: Release Rate = 0 kg/hr
#         3) N/A Events: Release Rate is N/A

canary_df = valid_event_df["Canary"].copy()
qube_df = valid_event_df["Qube"].copy()
sensirion_df = valid_event_df["Sensirion"].copy()
oiler_df = valid_event_df["Oiler"].copy()
soofie_df = valid_event_df["Soofie"].copy()


def classfy_for_sensors(df, sensor_name, offline_time=None):
    """
    Args:
        df: pd.DataFrame, data of sensors exclude soofie
        
    Returns:
        pd.DataFrame, classified data
    """

    if sensor_name != "Soofie":
        df.loc[df["Reported Release Rate (kg/h)"] > 0, "ReportLabel"] = "P"
        df.loc[df["Reported Release Rate (kg/h)"].isna(), "ReportLabel"] = "P"
        df.loc[df["Reported Release Rate (kg/h)"] == 0, "ReportLabel"] = "N"
        df.sort_values(by="EmissionStartDateTime", inplace=True)
        df.reset_index(drop=True, inplace=True)
    else:
        df.loc[df["Reported Release Rate (kg/h)"] > 0, "ReportLabel"] = "P"
        df.loc[df["Reported Release Rate (kg/h)"] == 0, "ReportLabel"] = "N"
        df.loc[df["Reported Release Rate (kg/h)"].isna(), "ReportLabel"] = str("NA")
        df.sort_values(by="EmissionStartDateTime", inplace=True)
        df.reset_index(drop=True, inplace=True)

    if offline_time is not None:
        for i in range(offline_time.shape[0]):
            start = offline_time.loc[i, "OfflineReportStartDateTime"]
            end = offline_time.loc[i, "OfflineReportEndDateTime"]
            idx = df.shape[0]
            df.loc[idx] = (
                {
                    "EmissionStartDateTime": start,
                    "EmissionEndDateTime": end,
                    "Reported Release Rate (kg/h)": pd.NA,
                    "EmissionRateUpper": pd.NA,
                    "EmissionRateLower": pd.NA,
                    "ReportLabel": str("NA"),
                }
            )
        df.reset_index(drop=True, inplace=True)

    # split events which start and end in different days
    print(sensor_name)
    for i in range(df.shape[0]):
        if df["EmissionStartDateTime"].dt.date[i] == df["EmissionEndDateTime"].dt.date[i]:
            continue
        else:
            df.loc[df.shape[0], :] = df.loc[i, :].copy()
            df.loc[df.shape[0] - 1, "EmissionStartDateTime"] = pd.to_datetime(str(df["EmissionEndDateTime"].dt.date[i]) + " 00:00:00")
            df.loc[i, "EmissionEndDateTime"] = pd.to_datetime(str(df["EmissionStartDateTime"].dt.date[i]) + " 23:59:59")

    df.sort_values(by="EmissionStartDateTime", inplace=True)
    df.reset_index(drop=True, inplace=True)

    # for every dates, if there is no event, then add 00:00:00~23:59:59 as a event with label "N"
    for date in pd.date_range(start=pd.to_datetime("2022-10-10"), end=pd.to_datetime("2022-11-30"), freq="D"):
        tmp_df = df.loc[df["EmissionStartDateTime"].dt.date == date, :]
        if tmp_df.shape[0] == 0:
            df.loc[df.shape[0]] = [date, date + pd.Timedelta(1, "d") - pd.Timedelta(1, "s"), 0, 0, 0, "N"]
            continue

        # if the start of first event is not 00:00:00, then add 00:00:00~start_time as a event with label "N"
        if tmp_df.loc[tmp_df.index[0], "EmissionStartDateTime"] != pd.to_datetime(str(date) + " 00:00:00"):
            df.loc[df.shape[0]] = [date, tmp_df.loc[tmp_df.index[0], "EmissionStartDateTime"] - pd.Timedelta(1, "s"), 0, 0, 0, "N"]
        
        # if the end of last event is not 23:59:59, then add end_time~23:59:59 as a event with label "N"
        if tmp_df.loc[tmp_df.index[-1], "EmissionEndDateTime"] != pd.to_datetime(str(date) + " 23:59:59"):
            df.loc[df.shape[0]] = [tmp_df.loc[tmp_df.index[-1], "EmissionEndDateTime"] + pd.Timedelta(1, "s"), 
                                   pd.to_datetime(str(date) + " 23:59:59"), 0, 0, 0, "N"]
            
        # for every two consecutive events, if the start of second event is not equal to the end of first event, then add a event with label "N"
        for i in range(tmp_df.shape[0] - 1):
            if tmp_df.loc[tmp_df.index[i], "EmissionEndDateTime"] != tmp_df.loc[tmp_df.index[i+1], "EmissionStartDateTime"]:
                df.loc[df.shape[0]] = [tmp_df.loc[tmp_df.index[i], "EmissionEndDateTime"] + pd.Timedelta(1, "s"), 
                                       tmp_df.loc[tmp_df.index[i+1], "EmissionStartDateTime"] - pd.Timedelta(1, "s"), 0, 0, 0, "N"]
    df.sort_values(by="EmissionStartDateTime", inplace=True)
    df.reset_index(drop=True, inplace=True)

    return df

andium_df = classfy_for_sensors(andium_df, "Andium")
ecoteco_df = classfy_for_sensors(ecoteco_df, "Ecoteco")
kuva_df = classfy_for_sensors(kuva_df, "Kuva", kuva_offline_time)
oiler_df = classfy_for_sensors(oiler_df, "Oiler", oiler_offline_time)
canary_df = classfy_for_sensors(canary_df, "Canary")
qube_df = classfy_for_sensors(qube_df, "Qube")
sensirion_df = classfy_for_sensors(sensirion_df, "Sensirion")
soofie_df = classfy_for_sensors(soofie_df, "Soofie")

andium_df["ReportDate"] = andium_df["EmissionStartDateTime"].dt.date
ecoteco_df["ReportDate"] = ecoteco_df["EmissionStartDateTime"].dt.date
kuva_df["ReportDate"] = kuva_df["EmissionStartDateTime"].dt.date
oiler_df["ReportDate"] = oiler_df["EmissionStartDateTime"].dt.date
canary_df["ReportDate"] = canary_df["EmissionStartDateTime"].dt.date
qube_df["ReportDate"] = qube_df["EmissionStartDateTime"].dt.date
sensirion_df["ReportDate"] = sensirion_df["EmissionStartDateTime"].dt.date
soofie_df["ReportDate"] = soofie_df["EmissionStartDateTime"].dt.date

# 3. Event filtering:
#     1) Canary: Short Stack Head Filtering, Exclude events that are not within the dates of October 31, 2022, and November 15 to November 30, 2022.
#     2) Kuva: Exclude events overlapping with Offline Report Dates
#     3) Oiler: Exclude events overlapping with Offline Report Dates
#     4) SOOFIE: Exclude events falling within the period of November 7, 2022, 22:30:00 to November 14, 2022, 16:30:00

# For Canary, only use data in short stack head
canary_df = canary_df[(canary_df["ReportDate"] == pd.to_datetime("2022-10-31").date()) | 
                      ((canary_df["ReportDate"] >= pd.to_datetime("2022-11-15").date()) & (canary_df["ReportDate"] <= pd.to_datetime("2022-11-30").date()))]

# For Soofie
soofie_df = soofie_df[~((soofie_df["EmissionStartDateTime"] >= pd.to_datetime("2022-11-07 22:30:00")) & (soofie_df["EmissionEndDateTime"] <= pd.to_datetime("2022-11-14 16:30:00")))]


# 4. For each sensor, filter out all events that do not overlap with the sensor's Online Date
# For Andium
andium_df = andium_df[(andium_df["EmissionStartDateTime"] >= online_date.loc[online_date["Sensor"] == "Andium", "Testing Beginning Date"].values[0]) & 
                      (andium_df["EmissionEndDateTime"] <= online_date.loc[online_date["Sensor"] == "Andium", "Testing Ending Date"].values[0])]
# For Ecoteco
ecoteco_df = ecoteco_df[(ecoteco_df["EmissionStartDateTime"] >= online_date.loc[online_date["Sensor"] == "Ecoteco", "Testing Beginning Date"].values[0]) & 
                        (ecoteco_df["EmissionEndDateTime"] <= online_date.loc[online_date["Sensor"] == "Ecoteco", "Testing Ending Date"].values[0])]
# For Kuva
kuva_df = kuva_df[(kuva_df["EmissionStartDateTime"] >= online_date.loc[online_date["Sensor"] == "Kuva", "Testing Beginning Date"].values[0]) & 
                  (kuva_df["EmissionEndDateTime"] <= online_date.loc[online_date["Sensor"] == "Kuva", "Testing Ending Date"].values[0])]
# For Oiler
oiler_df = oiler_df[(oiler_df["EmissionStartDateTime"] >= online_date.loc[online_date["Sensor"] == "Oiler", "Testing Beginning Date"].values[0]) & 
                    (oiler_df["EmissionEndDateTime"] <= online_date.loc[online_date["Sensor"] == "Oiler", "Testing Ending Date"].values[0])]
# For Canary
canary_df = canary_df[(canary_df["EmissionStartDateTime"] >= online_date.loc[online_date["Sensor"] == "Canary", "Testing Beginning Date"].values[0]) & 
                      (canary_df["EmissionEndDateTime"] <= online_date.loc[online_date["Sensor"] == "Canary", "Testing Ending Date"].values[0])]
# For Qube
qube_df = qube_df[(qube_df["EmissionStartDateTime"] >= online_date.loc[online_date["Sensor"] == "Qube", "Testing Beginning Date"].values[0]) & 
                  (qube_df["EmissionEndDateTime"] <= online_date.loc[online_date["Sensor"] == "Qube", "Testing Ending Date"].values[0])]
# For Sensirion
sensirion_df = sensirion_df[(sensirion_df["EmissionStartDateTime"] >= online_date.loc[online_date["Sensor"] == "Sensirion", "Testing Beginning Date"].values[0]) & 
                            (sensirion_df["EmissionEndDateTime"] <= online_date.loc[online_date["Sensor"] == "Sensirion", "Testing Ending Date"].values[0])]
# For Soofie
soofie_df = soofie_df[(soofie_df["EmissionStartDateTime"] >= online_date.loc[online_date["Sensor"] == "Soofie", "Testing Beginning Date"].values[0]) & 
                      (soofie_df["EmissionEndDateTime"] <= online_date.loc[online_date["Sensor"] == "Soofie", "Testing Ending Date"].values[0])]


# 5. Event Removal: 
#     - Exclude reports on: 2022-10-14, 2022-10-18 to 2022-10-22
#     - Reason: Stanford internal testing period


# For Andium
andium_df = andium_df[(andium_df["EmissionStartDateTime"].dt.date != pd.to_datetime("2022-10-14").date()) &
                        ((andium_df["EmissionStartDateTime"].dt.date < pd.to_datetime("2022-10-18").date()) | (andium_df["EmissionStartDateTime"].dt.date > pd.to_datetime("2022-10-22").date()))]
# For Ecoteco
ecoteco_df = ecoteco_df[(ecoteco_df["EmissionStartDateTime"].dt.date != pd.to_datetime("2022-10-14").date()) &
                        ((ecoteco_df["EmissionStartDateTime"].dt.date < pd.to_datetime("2022-10-18").date()) | (ecoteco_df["EmissionStartDateTime"].dt.date > pd.to_datetime("2022-10-22").date()))]
# For Kuva
kuva_df = kuva_df[(kuva_df["EmissionStartDateTime"].dt.date != pd.to_datetime("2022-10-14").date()) &
                  ((kuva_df["EmissionStartDateTime"].dt.date < pd.to_datetime("2022-10-18").date()) | (kuva_df["EmissionStartDateTime"].dt.date > pd.to_datetime("2022-10-22").date()))]
# For Oiler
oiler_df = oiler_df[(oiler_df["EmissionStartDateTime"].dt.date != pd.to_datetime("2022-10-14").date()) &
                    ((oiler_df["EmissionStartDateTime"].dt.date < pd.to_datetime("2022-10-18").date()) | (oiler_df["EmissionStartDateTime"].dt.date > pd.to_datetime("2022-10-22").date()))]
# For Canary
canary_df = canary_df[(canary_df["EmissionStartDateTime"].dt.date != pd.to_datetime("2022-10-14").date()) &
                      ((canary_df["EmissionStartDateTime"].dt.date < pd.to_datetime("2022-10-18").date()) | (canary_df["EmissionStartDateTime"].dt.date > pd.to_datetime("2022-10-22").date()))]
# For Qube
qube_df = qube_df[(qube_df["EmissionStartDateTime"].dt.date != pd.to_datetime("2022-10-14").date()) &
                  ((qube_df["EmissionStartDateTime"].dt.date < pd.to_datetime("2022-10-18").date()) | (qube_df["EmissionStartDateTime"].dt.date > pd.to_datetime("2022-10-22").date()))]
# For Sensirion
sensirion_df = sensirion_df[(sensirion_df["EmissionStartDateTime"].dt.date != pd.to_datetime("2022-10-14").date()) &
                            ((sensirion_df["EmissionStartDateTime"].dt.date < pd.to_datetime("2022-10-18").date()) | (sensirion_df["EmissionStartDateTime"].dt.date > pd.to_datetime("2022-10-22").date()))]
# For Soofie
soofie_df = soofie_df[(soofie_df["EmissionStartDateTime"].dt.date != pd.to_datetime("2022-10-14").date()) &
                      ((soofie_df["EmissionStartDateTime"].dt.date < pd.to_datetime("2022-10-18").date()) | (soofie_df["EmissionStartDateTime"].dt.date > pd.to_datetime("2022-10-22").date()))]


Andium
Ecoteco
Kuva
Oiler
Canary
Qube
Sensirion
Soofie


## 1.2.5 Save Team Data

In [11]:
# For Andium
save_path = PurePath('../../assets/sensor_data/Andium_validdata.csv')
andium_df.to_csv(save_path, index=None)

# For Ecoteco
save_path = PurePath('../../assets/sensor_data/Ecoteco_validdata.csv')
ecoteco_df.to_csv(save_path, index=None)

# For Kuva
save_path = PurePath('../../assets/sensor_data/Kuva_validdata.csv')
kuva_df.to_csv(save_path, index=None)


# For Oiler
save_path = PurePath('../../assets/sensor_data/Oiler_validdata.csv')
oiler_df.to_csv(save_path, index=None)

# For Canary
save_path = PurePath('../../assets/sensor_data/Canary_validdata.csv')
canary_df.to_csv(save_path, index=None)


# For Qube
save_path = PurePath('../../assets/sensor_data/Qube_validdata.csv')
qube_df.to_csv(save_path, index=None)

# For Sensirion
save_path = PurePath('../../assets/sensor_data/Sensirion_validdata.csv')
sensirion_df.to_csv(save_path, index=None)

# For Soofie
save_path = PurePath('../../assets/sensor_data/Soofie_validdata.csv')
soofie_df.to_csv(save_path, index=None)